# Installing Library

In [1]:
!pip install transformers datasets evaluate sacrebleu numpy 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 761.4 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 4.6 MB/s eta 0:00:00


# Loading Library

In [2]:
from transformers import MarianTokenizer, MarianMTModel, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from datasets import load_dataset
import evaluate
import numpy

# Loading Data


#### Data: https://huggingface.co/datasets/Helsinki-NLP/news_commentary
#### Model: https://huggingface.co/Helsinki-NLP/opus-mt-ar-it

In [3]:
ds = load_dataset("Helsinki-NLP/news_commentary", "ar-it")
ds = ds.remove_columns('id')
ds

Generating train split:   0%|          | 0/17227 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 17227
    })
})

In [4]:
ds = ds['train'].train_test_split(train_size=0.8)
ds

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 13781
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 3446
    })
})

# Loading Model & Tokenizer

In [5]:
checkpoints = 'Helsinki-NLP/opus-mt-ar-it'
model = MarianMTModel.from_pretrained(checkpoints)
tokenizer = MarianTokenizer.from_pretrained(checkpoints)

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/309M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/899k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/824k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Make Preprocessing In Text

In [6]:
ds

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 13781
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 3446
    })
})

In [7]:
ds['train'][0]

{'translation': {'ar': 'وفي التحدي الأكثر إلحاحاً الذي يواجه الاتحاد الأوروبي، والذي تفرضه روسيا، سوف يضطر تاسك إلى التوسط في القرارات مع القادة من البلدان التي تشعر بأنها مهددة بشكل مباشر (مثل بلده) وتلك التي تفوق أهمية علاقاتها الاقتصادية مع روسيا أي تهديد لأمن الأوروبي، والذي تشعر بأنه بعيد عنها على أية حال. وعلى جبهة الاقتصاد، يتعين عليه أن يوفق بين أولويات ألمانيا حيث التشغيل الكامل للعمالة وأولويات بلدان مثل اليونان وإيطاليا التي لا تزال واقعة في قبضة الركود ومعدلات البطالة المرتفعة إلى عنان السماء. وقد تكون القدرة على التحدث بشكل مباشر مع أعضاء المجلس، باللغة الإنجليزية في الأغلب، بمثابة التحدي المباشر الأعظم كما اعترف هو شخصيا.',
  'it': 'Per quanto riguarda la sfida più immediata dell’Ue, posta dalla Russia, Tusk dovrà mediare le decisioni con i leader provenienti dai Paesi che si sentono immediatamente minacciati (come il suo) e con quelli per cui i legami economici con la Russia superano qualsiasi minaccia alla sicurezza europea, fatto che sentono essere remoto. Sul fronte e

In [8]:
source_lang = 'ar'
target_lang = 'it'

def preprocessing(batch):
    inputs = [example[source_lang] for example in batch['translation']]
    targets = [example[target_lang] for example in batch['translation']]
    model_inputs = tokenizer(inputs, text_target=targets, padding=True,
                             return_tensors='pt', truncation=True)
    
    return model_inputs

In [9]:
ds = ds.map(preprocessing, batched=True)

Map:   0%|          | 0/13781 [00:00<?, ? examples/s]

Map:   0%|          | 0/3446 [00:00<?, ? examples/s]

In [10]:
ds

DatasetDict({
    train: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 13781
    })
    test: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3446
    })
})

# DataCollatorForSeq2Seq

In [11]:
data_collector = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoints)

# Create Compute Metrics

In [12]:
metric = evaluate.load("sacrebleu")

In [13]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

# Create Arguments

In [21]:
model_args = Seq2SeqTrainingArguments(
    output_dir="./Helsinki-mt-ar-it",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    num_train_epochs=3,
    fp16=True,
    warmup_steps=100,
    logging_steps=100, save_steps=4000
)

In [22]:
trainer = Seq2SeqTrainer(
    model=model,
    args=model_args,
    tokenizer=tokenizer,
    data_collator=data_collector,
    compute_metrics=compute_metrics,
    train_dataset=ds['train'],
    eval_dataset=ds['test']
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


# Training Model

In [23]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
100,0.744200
200,0.727800
300,0.725900
400,0.830300
500,0.812500
600,0.812200
700,0.824400
800,0.822300
900,0.777200
1000,0.752200


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[63293]], 'forced_eos_token_id': 0}


TrainOutput(global_step=2586, training_loss=0.7518522479417983, metrics={'train_runtime': 1060.5428, 'train_samples_per_second': 38.983, 'train_steps_per_second': 2.438, 'total_flos': 2422072866963456.0, 'train_loss': 0.7518522479417983, 'epoch': 3.0})

# Saving Model

In [24]:
trainer.save_model(r'/kaggle/working/model')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[63293]], 'forced_eos_token_id': 0}


# Make Prediction

In [28]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

In [29]:
def predict(text, model_checkpoint):
    print('Input: ', text, '\n\n')
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    inputs = tokenizer(text, return_tensors="pt").input_ids
    model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
    outputs = model.generate(inputs, max_new_tokens=300, do_sample=True, top_k=30, top_p=0.95)
    return "Output:  " + tokenizer.decode(outputs[0], skip_special_tokens=True)

In [30]:
text = 'مرحبا'
predict(text, '/kaggle/working/Helsinki-mt-ar-it/checkpoint-2586')

Input:  مرحبا 




/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


"Output:  C'e' nessuno?"

# Create Pipeline

In [32]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

model_path = r'/kaggle/working/model'
# model_path = r'/kaggle/working/Helsinki-mt-ar-it/checkpoint-2586'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

# Create a translation pipeline
translation_pipeline = pipeline('translation', model=model, tokenizer=tokenizer)

# Example usage
input_text = "مرحبا"
translated_text = translation_pipeline(input_text)[0]['translation_text']

# Output the translated text
print(translated_text)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


C'e' nessuno?


# Deploy Model

In [37]:
!pip install streamlit

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 28.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 73.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.4 MB/s eta 0:00:00


In [44]:
import streamlit as st
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

model_path = r'/kaggle/working/model'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

translation_pipeline = pipeline('translation', model=model, tokenizer=tokenizer)

st.title("Translation Model")

input_text = st.text_area("Enter text to translate:")

if st.button("Translate"):
    if input_text:
        translated_text = translation_pipeline(input_text)
        st.write("Translation:", translated_text[0]['translation_text'])
    else:
        st.write("Please enter some text.")